# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [286]:
import pandas as pd
import numpy as np

In [287]:
df = pd.read_csv('data/bank-additional/bank-additional-full.csv', sep = ';')

In [288]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



#### 'unknown' values contain missing data.  We will need to address them as missing data.
- job
- marital
- education
- default
- housing
- loan

In [289]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

#### 'object' data types will need to be coerced to categorical data types.
- job
- marital
- education
- default
- housing
- loan
- contact
- month
- day_of_week
- poutcome
- y (target variable)

#### 'y' column
- to be renamed to 'target' for clarity.
- to be converted to binary 0/1 values for modeling.

#### 'pdays' column should be binary feature

In [290]:
#### Data Cleaning

In [291]:
# rename 'y' column to 'target'
df = df.rename(columns={'y': 'target'})

In [292]:
# convert 'target' column to binary 0/1 values
df['target'] = df['target'].map({'yes': 1, 'no': 0})
df.head()

# get all unique values of target column
df['target'].unique()

array([0, 1])

In [293]:
# convert 'pdays' column to binary feature
df['pdays'] = df['pdays'].apply(lambda x: 0 if x == 999 else 1)

# get all unique values of pdays column
df['pdays'].unique()

array([0, 1])

In [294]:
# List of categorical columns to one-hot encode
categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']

# Loop through the columns and apply one-hot encoding
for col in categorical_columns:
    dummies = pd.get_dummies(df[col], prefix=col)
    df = pd.concat([df, dummies], axis=1)
    df.drop(col, axis=1, inplace=True)


In [295]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 64 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   age                            41188 non-null  int64  
 1   duration                       41188 non-null  int64  
 2   campaign                       41188 non-null  int64  
 3   pdays                          41188 non-null  int64  
 4   previous                       41188 non-null  int64  
 5   emp.var.rate                   41188 non-null  float64
 6   cons.price.idx                 41188 non-null  float64
 7   cons.conf.idx                  41188 non-null  float64
 8   euribor3m                      41188 non-null  float64
 9   nr.employed                    41188 non-null  float64
 10  target                         41188 non-null  int64  
 11  job_admin.                     41188 non-null  bool   
 12  job_blue-collar                41188 non-null 

In [296]:
# Save the cleaned data to a new CSV file
df.to_csv('data/bank-additional/bank-additional-cleaned.csv', index=False)

df = pd.read_csv('data/bank-additional/bank-additional-cleaned.csv')


### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [297]:
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 64 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   age                            41188 non-null  int64  
 1   duration                       41188 non-null  int64  
 2   campaign                       41188 non-null  int64  
 3   pdays                          41188 non-null  int64  
 4   previous                       41188 non-null  int64  
 5   emp.var.rate                   41188 non-null  float64
 6   cons.price.idx                 41188 non-null  float64
 7   cons.conf.idx                  41188 non-null  float64
 8   euribor3m                      41188 non-null  float64
 9   nr.employed                    41188 non-null  float64
 10  target                         41188 non-null  int64  
 11  job_admin.                     41188 non-null  bool   
 12  job_blue-collar                41188 non-null 

#### Business Objective

The business objective is to develop a predictive model that identifies which clients are most likely to subscribe to a term deposit when contacted during a direct marketing campaign.

By accurately predicting customer response, the bank can focus its marketing efforts on the most promising leads, thereby increasing the campaign's effectiveness, improving the conversion rate, and maximizing the return on investment.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [298]:
# Define features (X) and target (y)
# Bank client data features
bank_features = ['age'] + [col for col in df.columns if col.startswith(('job_', 'marital_', 'education_', 'default_', 'housing_', 'loan_'))]

X = df[bank_features]
y = df['target']

# Display the first few rows of X and the shape of X and y
print("Features (X):")
print(X.head())
print("\nShape of X:", X.shape)
print("Shape of y:", y.shape)

Features (X):
   age  job_admin.  job_blue-collar  job_entrepreneur  job_housemaid  \
0   56       False            False             False           True   
1   57       False            False             False          False   
2   37       False            False             False          False   
3   40        True            False             False          False   
4   56       False            False             False          False   

   job_management  job_retired  job_self-employed  job_services  job_student  \
0           False        False              False         False        False   
1           False        False              False          True        False   
2           False        False              False          True        False   
3           False        False              False         False        False   
4           False        False              False          True        False   

   ...  education_unknown  default_no  default_unknown  default_yes  \
0

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [299]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Print the shapes of the resulting datasets
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (32950, 34)
Shape of X_test: (8238, 34)
Shape of y_train: (32950,)
Shape of y_test: (8238,)


### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [300]:
# Calculate the baseline accuracy by predicting the majority class
baseline_accuracy = y_train.value_counts(normalize=True).max()

print(f"The baseline accuracy is: {baseline_accuracy:.4f}")
print("This means that if we always predict the majority class (no subscription), we would be correct about 88.73% of the time. Our models must beat this score.")

The baseline accuracy is: 0.8873
This means that if we always predict the majority class (no subscription), we would be correct about 88.73% of the time. Our models must beat this score.


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.

In [301]:
from sklearn.linear_model import LogisticRegression

# Instantiate the model
log_reg = LogisticRegression(random_state=42)

# Fit the model to the training data
log_reg.fit(X_train, y_train)

/Users/joambrose/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


### Problem 9: Score the Model

What is the accuracy of your model?

In [302]:
# Score the model on the test data
accuracy = log_reg.score(X_test, y_test)

print(f"The accuracy of the Logistic Regression model is: {accuracy:.4f}")

The accuracy of the Logistic Regression model is: 0.8874


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [303]:
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Re-initialize Logistic Regression to include it in the comparison loop
log_reg_comp = LogisticRegression(random_state=42, max_iter=1000) # Increased max_iter to avoid convergence warnings

models = {
    "Logistic Regression": log_reg_comp,
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Support Vector Machine": SVC(random_state=42)
}

results = []

for model_name, model in models.items():
    # Time the training
    start_time = time.time()
    model.fit(X_train, y_train)
    end_time = time.time()
    train_time = end_time - start_time

    # Calculate accuracies
    train_accuracy = model.score(X_train, y_train)
    test_accuracy = model.score(X_test, y_test)

    results.append({
        "Model": model_name,
        "Train Time": train_time,
        "Train Accuracy": train_accuracy,
        "Test Accuracy": test_accuracy
    })

results_df = pd.DataFrame(results)
print(results_df)

                    Model  Train Time  Train Accuracy  Test Accuracy
0     Logistic Regression    0.127772        0.887344       0.887351
1     K-Nearest Neighbors    0.002681        0.891593       0.876912
2           Decision Tree    0.068905        0.917056       0.864773
3  Support Vector Machine    5.063323        0.887344       0.887351


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [304]:
# --- 1. More Feature Engineering ---
print("--- Using All Features ---")

# Define features (X) and target (y) using all available data
# We exclude 'duration' as it's not known before a call is made
X_all = df.drop(columns=['target', 'duration'])
y_all = df['target']

# Split the data into training and testing sets
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X_all, y_all, test_size=0.2, random_state=42, stratify=y_all)

# Rerun model comparison with all features
results_all = []
for model_name, model in models.items():
    start_time = time.time()
    model.fit(X_train_all, y_train_all)
    end_time = time.time()
    train_time = end_time - start_time

    train_accuracy = model.score(X_train_all, y_train_all)
    test_accuracy = model.score(X_test_all, y_test_all)

    results_all.append({
        "Model": model_name,
        "Train Time": train_time,
        "Train Accuracy": train_accuracy,
        "Test Accuracy": test_accuracy
    })

results_all_df = pd.DataFrame(results_all)
print(results_all_df)

--- Using All Features ---


/Users/joambrose/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                    Model  Train Time  Train Accuracy  Test Accuracy
0     Logistic Regression    1.319559        0.899727       0.900583
1     K-Nearest Neighbors    0.005311        0.911411       0.894149
2           Decision Tree    0.155248        0.995387       0.838553
3  Support Vector Machine    5.053838        0.887344       0.887351


In [305]:
# --- 2. Hyperparameter Tuning (Grid Search) ---
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

print("\n--- Hyperparameter Tuning for Decision Tree ---")

# Define the parameter grid
param_grid = {
    'max_depth': [3, 5, 7, 10, None],
    'min_samples_leaf': [1, 5, 10],
    'criterion': ['gini', 'entropy']
}

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=5,
                           n_jobs=-1,
                           verbose=1,
                           scoring='accuracy')


--- Hyperparameter Tuning for Decision Tree ---


In [306]:
# Fit the grid search to the data
grid_search.fit(X_train_all, y_train_all)

# Print the best parameters and the best score
print("\nBest Parameters found: ", grid_search.best_params_)
print("Best accuracy from Grid Search: ", grid_search.best_score_)

# Evaluate the best model on the test set
best_tree = grid_search.best_estimator_
test_accuracy = best_tree.score(X_test_all, y_test_all)
print("Test Accuracy of Tuned Decision Tree: ", test_accuracy)


Fitting 5 folds for each of 30 candidates, totalling 150 fits

Best Parameters found:  {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 10}
Best accuracy from Grid Search:  0.8997875569044005
Test Accuracy of Tuned Decision Tree:  0.9020393299344501
